# Analysis

**Initializing**

In [38]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.simplefilter(action='ignore', category=pd.errors.SettingWithCopyWarning)

from Brands import Brands
from Analyzer import Analyzer
from Solver import Solver

# Set the aesthetic style of the plots
sns.set(style="whitegrid")

#an = Analyzer(marketing_mix_segment_weights=move_mkt_weights, last_period=2)
an = Analyzer(last_period=7, sector="Vodites")

brd = Brands(sector="Vodites")

solver = Solver(sector="Vodites")


Vodites
./Exports/TeamExport_A46051_Alpha_M_Period 7.xlsx loaded
No weights provided
Vodites
./Exports/TeamExport_A46051_Alpha_M_Period 7.xlsx loaded
Attributes file:./Attributes/Vodites/attributes_Vodites_7.json


**Percentage expenditure by segment for each product**

In [39]:
df_marketing_mixes = brd.get_marketing_mixes(capped=False)
df_marketing_mixes


,Innovators,Early Adopters,Followers
0,0.150000,0.482857,0.367143
1,0.216471,0.230588,0.552941
2,0.137931,0.137931,0.724138


## Semantic scales

In [40]:
features_name = ["Resolution", "Energy Efficiency", "Carbon Footprint", "Connectivity", "No. of Apps", "Price"]
df_seg_sem = brd.df_segments_semantic[:3][features_name]
df_seg_sem


,Resolution,Energy Efficiency,Carbon Footprint,Connectivity,No. of Apps,Price
Index,,,,,,
Adopters_7,4.98,4.28,4.73,4.63,4.44,5.16
Followers_7,4.37,4.76,2.78,3.88,3.77,4.48
Innovators_7,5.38,3.7,4.07,5.23,3.87,5.5


In [41]:
brd.df_brands_semantic

,Resolution,Energy Efficiency,Carbon Footprint,Connectivity,No. of Apps,Price
MARKET : Vodites,,,,,,
MEOW,4.29,2.94,4.56,4.52,4.14,5.37
REEGO,5.24,2.81,4.41,4.86,3.64,5.33
TENOR,2.91,3.36,6.39,2.89,1.58,4.83


### Compute Distances between Segments and Products 

#### Compute closest brands for each segment

In [42]:
brd.df_brands_semantic

,Resolution,Energy Efficiency,Carbon Footprint,Connectivity,No. of Apps,Price
MARKET : Vodites,,,,,,
MEOW,4.29,2.94,4.56,4.52,4.14,5.37
REEGO,5.24,2.81,4.41,4.86,3.64,5.33
TENOR,2.91,3.36,6.39,2.89,1.58,4.83


In [43]:
_ = an.get_n_closest(df_base=df_seg_sem, df_performers=brd.df_brands_semantic, num_top=3, max_distance_1D=6, distance_metric=3)

---------- Adopters_7 ----------
Segment:	 MEOW
Distance:	 0.8992884384618579
Segment:	 REEGO
Distance:	 0.8934623559118977
Segment:	 TENOR
Distance:	 0.727070195646252

---------- Followers_7 ----------
Segment:	 MEOW
Distance:	 0.8331312499640449
Segment:	 REEGO
Distance:	 0.8095888521415876
Segment:	 TENOR
Distance:	 0.7507849578077158

---------- Innovators_7 ----------
Segment:	 REEGO
Distance:	 0.9299268006440846
Segment:	 MEOW
Distance:	 0.886517003375502
Segment:	 TENOR
Distance:	 0.6836656413372264



#### Compute closest Segments for each Brand

In [44]:
_ = an.get_n_closest(df_base=brd.df_brands_semantic, df_performers=df_seg_sem, max_distance_1D=6)

---------- MEOW ----------
Segment:	 Adopters_7
Distance:	 0.8992884384618579
Segment:	 Innovators_7
Distance:	 0.886517003375502
Segment:	 Followers_7
Distance:	 0.8331312499640449

---------- REEGO ----------
Segment:	 Innovators_7
Distance:	 0.9299268006440846
Segment:	 Adopters_7
Distance:	 0.8934623559118977
Segment:	 Followers_7
Distance:	 0.8095888521415876

---------- TENOR ----------
Segment:	 Followers_7
Distance:	 0.7507849578077158
Segment:	 Adopters_7
Distance:	 0.727070195646252
Segment:	 Innovators_7
Distance:	 0.6836656413372264



#### Distances between all points

In [45]:
df_all_sem = brd.get_comprehensive_df_semantic()
index = df_all_sem.index 

distances = an.compute_distance_centroids(df_all_sem, df_all_sem, max_distance_1D=6)[3]


df_out_sem = pd.DataFrame(columns=index, index=index)

for i, start in enumerate(index):
    index_to_search = index[i:]
    for stop in index_to_search:
        df_out_sem.loc[start, stop] = distances[start][stop]
        df_out_sem.loc[stop, start] = distances[start][stop]

df_out_sem

,MEOW,REEGO,TENOR,Adopters_7,Followers_7,Innovators_7
MEOW,1.0,0.922372,0.764245,0.899288,0.833131,0.886517
REEGO,0.922372,1.0,0.720415,0.893462,0.809589,0.929927
TENOR,0.764245,0.720415,1.0,0.72707,0.750785,0.683666
Adopters_7,0.899288,0.893462,0.72707,1.0,0.864839,0.914012
Followers_7,0.833131,0.809589,0.750785,0.864839,1.0,0.815283
Innovators_7,0.886517,0.929927,0.683666,0.914012,0.815283,1.0


## Multi Dimensional Scaling

In [46]:
brd.df_segments_mds

,Segment,Period,Autonomy,Sophistication,Economy
Index,,,,,
Adopters_7,Adopters,7,3.52,3.94,-7.76
Followers_7,Followers,7,0.96,2.32,-3.2
Innovators_7,Innovators,7,5.16,5.24,-9.98
Early Adopters_6,Adopters,6,2.62,3.46,-8.38
Followers_6,Followers,6,0.38,1,-3.56
Innovators_6,Innovators,6,4.44,4.8,-12.24
Early Adopters_5,Adopters,5,1.46,2.76,-8.82
Followers_5,Followers,5,-0.16,-0.5,-3.92
Innovators_5,Innovators,5,3.72,4.36,-13.7


In [47]:
brd.df_brands_mds

,Autonomy,Sophistication,Economy
MARKET : Vodites,,,
MEOW,0.28,0.82,-9.14
REEGO,1.64,3.96,-8.88
TENOR,-6.48,-10.80,-5.56


### Compute Distances between Segments and Products 

#### Compute closest brands for each segment

In [48]:
brd.df_segments_mds

,Segment,Period,Autonomy,Sophistication,Economy
Index,,,,,
Adopters_7,Adopters,7,3.52,3.94,-7.76
Followers_7,Followers,7,0.96,2.32,-3.2
Innovators_7,Innovators,7,5.16,5.24,-9.98
Early Adopters_6,Adopters,6,2.62,3.46,-8.38
Followers_6,Followers,6,0.38,1,-3.56
Innovators_6,Innovators,6,4.44,4.8,-12.24
Early Adopters_5,Adopters,5,1.46,2.76,-8.82
Followers_5,Followers,5,-0.16,-0.5,-3.92
Innovators_5,Innovators,5,3.72,4.36,-13.7


In [49]:
df_seg_mds = brd.df_segments_mds[:3][['Autonomy', 'Sophistication', 'Economy']]

_ = an.get_n_closest(df_base=df_seg_mds, df_performers=brd.df_brands_mds, num_top=3, max_distance_1D=40, weighted="eq")

---------- Adopters_7 ----------
Segment:	 REEGO
Distance:	 0.9684128190558258
Segment:	 MEOW
Distance:	 0.9320901332647457
Segment:	 TENOR
Distance:	 0.7409522244835907

---------- Followers_7 ----------
Segment:	 REEGO
Distance:	 0.9141047149140303
Segment:	 MEOW
Distance:	 0.9110290309520385
Segment:	 TENOR
Distance:	 0.7796510343417363

---------- Innovators_7 ----------
Segment:	 REEGO
Distance:	 0.9436550800870211
Segment:	 MEOW
Distance:	 0.9041959465019007
Segment:	 TENOR
Distance:	 0.7069173893478724



#### Compute closest Segments for each Brand

In [50]:
_ = an.get_n_closest(df_base=brd.df_brands_mds, df_performers=df_seg_mds, num_top=3, max_distance_1D=40, weighted="eq")

---------- MEOW ----------
Segment:	 Adopters_7
Distance:	 0.9320901332647457
Segment:	 Followers_7
Distance:	 0.9110290309520385
Segment:	 Innovators_7
Distance:	 0.9041959465019007

---------- REEGO ----------
Segment:	 Adopters_7
Distance:	 0.9684128190558258
Segment:	 Innovators_7
Distance:	 0.9436550800870211
Segment:	 Followers_7
Distance:	 0.9141047149140303

---------- TENOR ----------
Segment:	 Followers_7
Distance:	 0.7796510343417363
Segment:	 Adopters_7
Distance:	 0.7409522244835907
Segment:	 Innovators_7
Distance:	 0.7069173893478724



#### Distances between all points

In [51]:
df_all_mds = brd.get_comprehensive_df_mds()
index = df_all_mds.index 

distances = an.compute_distance_centroids(df_all_mds, df_all_mds, weighted="eq", max_distance_1D=40)[3]


df_out_mds = pd.DataFrame(columns=index, index=index)

for i, start in enumerate(index):
    index_to_search = index[i:]
    for stop in index_to_search:
        df_out_mds.loc[start, stop] = distances[start][stop]
        df_out_mds.loc[stop, start] = distances[start][stop]

df_out_mds

,MEOW,REEGO,TENOR,Adopters_7,Followers_7,Innovators_7
MEOW,1.0,0.950467,0.7992,0.93209,0.911029,0.904196
REEGO,0.950467,1.0,0.75217,0.968413,0.914105,0.943655
TENOR,0.7992,0.75217,1.0,0.740952,0.779651,0.706917
Adopters_7,0.93209,0.968413,0.740952,1.0,0.92098,0.955964
Followers_7,0.911029,0.914105,0.779651,0.92098,1.0,0.877411
Innovators_7,0.904196,0.943655,0.706917,0.955964,0.877411,1.0


# Forecast analysis

In [52]:
df_seg_sem_fc = an.forecast_df(brd.df_segments_semantic, steps=2)

df_seg_sem_fc = df_seg_sem_fc[(df_seg_sem_fc["Period"]>an.last_period)].drop(labels=["Period", "Segment"], axis=1)
df_seg_sem_fc

,Resolution,Energy Efficiency,Carbon Footprint,Connectivity,No. of Apps,Price
Innovators_9,5.52,3.96,3.93,5.43,3.95,4.82
Adopters_9,5.18,4.38,4.67,4.95,4.46,4.96
Followers_9,4.85,4.84,2.44,4.1,4.01,4.38
Innovators_8,5.45,3.83,4.0,5.33,3.91,5.16
Followers_8,4.61,4.8,2.61,3.99,3.89,4.43
Adopters_8,5.08,4.33,4.7,4.79,4.45,5.06


In [53]:
df_all_sem = pd.concat([df_seg_sem_fc, brd.df_brands_semantic], join="inner")
index = df_all_sem.index

distances = an.compute_distance_centroids(df_all_sem, df_all_sem, weighted="default", max_distance_1D=6)[3]


df_out_sem_fc = pd.DataFrame(columns=index, index=index)

for i, start in enumerate(index):
    index_to_search = index[i:]
    for stop in index_to_search:
        df_out_sem_fc.loc[start, stop] = distances[start][stop]
        df_out_sem_fc.loc[stop, start] = distances[start][stop]

df_out_sem_fc

,Innovators_9,Adopters_9,Followers_9,Innovators_8,Followers_8,Adopters_8,MEOW,REEGO,TENOR
Innovators_9,1.0,0.930031,0.841948,0.96932,0.83299,0.917039,0.855079,0.896674,0.665888
Adopters_9,0.930031,1.0,0.862494,0.932533,0.858362,0.981708,0.87752,0.887681,0.700951
Followers_9,0.841948,0.862494,1.0,0.837045,0.977944,0.866347,0.82131,0.812493,0.712403
Innovators_8,0.96932,0.932533,0.837045,1.0,0.830311,0.924476,0.873513,0.917223,0.676105
Followers_8,0.83299,0.858362,0.977944,0.830311,1.0,0.864384,0.828532,0.812327,0.731814
Adopters_8,0.917039,0.981708,0.866347,0.924476,0.864384,1.0,0.889377,0.892073,0.714297
MEOW,0.855079,0.87752,0.82131,0.873513,0.828532,0.889377,1.0,0.922372,0.764245
REEGO,0.896674,0.887681,0.812493,0.917223,0.812327,0.892073,0.922372,1.0,0.720415
TENOR,0.665888,0.700951,0.712403,0.676105,0.731814,0.714297,0.764245,0.720415,1.0


In [54]:
df_seg_mds_fc = an.forecast_df(brd.df_segments_mds, steps=2)

df_seg_mds_fc = df_seg_mds_fc[(df_seg_mds_fc["Period"]>an.last_period)][['Autonomy', 'Sophistication', 'Economy']]
df_seg_mds_fc

,Autonomy,Sophistication,Economy
Innovators_9,6.6,6.12,-5.46
Adopters_9,5.32,4.9,-6.52
Followers_9,2.12,4.96,-2.48
Innovators_8,5.88,5.68,-7.72
Followers_8,1.54,3.64,-2.84
Adopters_8,4.42,4.42,-7.14


In [55]:
brd.df_brands_mds

,Autonomy,Sophistication,Economy
MARKET : Vodites,,,
MEOW,0.28,0.82,-9.14
REEGO,1.64,3.96,-8.88
TENOR,-6.48,-10.80,-5.56


In [56]:
df_all_mds = pd.concat([df_seg_mds_fc, brd.df_brands_mds], join="inner")
index = df_all_mds.index

distances = an.compute_distance_centroids(df_all_mds, df_all_mds, weighted="eq", max_distance_1D=40)[3]


df_out_mds_fc = pd.DataFrame(columns=index, index=index)

for i, start in enumerate(index):
    index_to_search = index[i:]
    for stop in index_to_search:
        df_out_mds_fc.loc[start, stop] = distances[start][stop]
        df_out_mds_fc.loc[stop, start] = distances[start][stop]

df_out_mds_fc

,Innovators_9,Adopters_9,Followers_9,Innovators_8,Followers_8,Adopters_8,MEOW,REEGO,TENOR
Innovators_9,1.0,0.970243,0.920553,0.96518,0.910303,0.953308,0.869636,0.90762,0.691312
Adopters_9,0.970243,1.0,0.925606,0.977817,0.921713,0.982771,0.899054,0.935457,0.716182
Followers_9,0.920553,0.925606,1.0,0.906332,0.97855,0.924588,0.883738,0.906247,0.737074
Innovators_8,0.96518,0.977817,0.906332,1.0,0.901246,0.970933,0.891031,0.931868,0.701035
Followers_8,0.910303,0.921713,0.97855,0.901246,1.0,0.924456,0.898727,0.912686,0.758377
Adopters_8,0.953308,0.982771,0.924588,0.970933,0.924456,1.0,0.915714,0.952199,0.728832
MEOW,0.869636,0.899054,0.883738,0.891031,0.898727,0.915714,1.0,0.950467,0.7992
REEGO,0.90762,0.935457,0.906247,0.931868,0.912686,0.952199,0.950467,1.0,0.75217
TENOR,0.691312,0.716182,0.737074,0.701035,0.758377,0.728832,0.7992,0.75217,1.0


## MEOW - Next Period

In [57]:
meow_feat = brd.get_features("MEOW").drop(labels="Base Cost", axis=1).values.flatten()

meow_feat

array([ 63,  35,  31,   7,  54, 671], dtype=int64)

In [58]:
from Utils import combined_error

target_sem = df_seg_sem_fc.loc[f"Adopters_{an.last_period+1}"]
target_mds = [0,0,0]


prices_array = np.linspace(650, 750, num=8)
features = meow_feat.copy()

for price in prices_array:
    price = round(price)
    print(f"Price {price}")
    no_price_features = np.delete(features, -1)
    new_features_meow = np.append(no_price_features, price)
    res = combined_error(features=new_features_meow, ideal_semantic=target_sem, ideal_mds=target_mds, semantic_weights=brd.rel_importance_features, mds_weights=[1/3, 1/3,1/3], error_weights=[1,0], model=solver)
    print(f"Combined_error: {res}")
    contr = an.compute_contribution(price, 206, [17/38,12/38,9/38]) ### CHANGE THIS
    print(f"Contribution: {contr}")
    print()



Price 650
semantic_error 0.1308086620653779 mds_error 0.0
Combined_error: 0.2616173241307558
Contribution: 202

Price 664
semantic_error 0.13132352451847173 mds_error 0.0
Combined_error: 0.26264704903694347
Contribution: 210

Price 679
semantic_error 0.132286289969469 mds_error 0.0
Combined_error: 0.264572579938938
Contribution: 220

Price 693
semantic_error 0.1335606606784906 mds_error 0.0
Combined_error: 0.2671213213569812
Contribution: 228

Price 707
semantic_error 0.13518741134991008 mds_error 0.0
Combined_error: 0.27037482269982016
Contribution: 237

Price 721
semantic_error 0.13715400407473044 mds_error 0.0
Combined_error: 0.2743080081494609
Contribution: 246

Price 736
semantic_error 0.1396218241648347 mds_error 0.0
Combined_error: 0.2792436483296694
Contribution: 255

Price 750
semantic_error 0.14224508418826443 mds_error 0.0
Combined_error: 0.28449016837652885
Contribution: 264



### Semantic Scales

In [59]:
new_features_meow = meow_feat # Change price

semantic_regressed_meow = np.array(solver.regress_semantic(new_features_meow))
semantic_regressed_meow = [round(sem, 1) for sem in semantic_regressed_meow]
semantic_regressed_meow

[4.3, 2.6, 4.6, 4.3, 4.1, 5.3]

In [60]:
res = an.compute_distance_centroids(semantic_regressed_meow, target_sem)
res[3]

{'observation': {'centroid': 0.8674469214712424}}

In [61]:
res[4]

{'observation': {'centroid': array([0.7800000000000011, 1.73, 0.10000000000000142, 0.4900000000000002,
         0.3500000000000014, 0.23999999999999932], dtype=object)}}

### Multi Dimensional Scaling

In [62]:
mds_regressed = np.array(solver.regress_mds(new_features_meow))
mds_regressed 

array([0, 0, 0], dtype=int64)

In [63]:
res = an.compute_distance_centroids(mds_regressed, target_mds, max_distance_1D=40, weighted="eq")
res[3]

d:\Uni\Anno III\UCSD\112\Markstrat\Code\Utils.py:271: RuntimeWarning: invalid value encountered in divide
  relative_distance = manhattan_distance / feat_values_array
d:\Uni\Anno III\UCSD\112\Markstrat\Code\Utils.py:283: RuntimeWarning: invalid value encountered in divide
  relative_metric = (current_centroid - feat_values_array)/feat_values_array


{'observation': {'centroid': 1.0}}

### Sensitivity Analysis

## MORE - Next Period

In [64]:
more_features = np.array([0,0,0,0,0,0])
#more_features = brd.get_features("MORE").drop(labels="Base Cost", axis=1).values.flatten()

In [65]:
from Utils import combined_error

explorers_semantic_mk = df_seg_sem_fc.loc[f"Followers_{an.last_period+1}"]
explorers_mds_mk = [0,0,0]

prices_array = np.linspace(650, 750, num=8)
features = more_features.copy()

for price in prices_array:
    price = round(price)
    print(f"Price {price}")
    no_price_features = np.delete(features, -1)
    new_features = np.append(no_price_features, price)
    res = combined_error(features=new_features, ideal_semantic=target_sem, ideal_mds=target_mds, semantic_weights=brd.rel_importance_features, mds_weights=[1/3, 1/3,1/3], error_weights=[1,0], model=solver)
    print(f"Combined_error: {res}")
    contr = an.compute_contribution(price, 206, [17/38,12/38,9/38]) ### CHANGE THIS
    print(f"Contribution: {contr}")
    print()



Price 650
semantic_error 0.7307739835201295 mds_error 0.0
Combined_error: 1.461547967040259
Contribution: 202

Price 664
semantic_error 0.7308663195212226 mds_error 0.0
Combined_error: 1.4617326390424452
Contribution: 210

Price 679
semantic_error 0.7310399246501046 mds_error 0.0
Combined_error: 1.4620798493002092
Contribution: 220

Price 693
semantic_error 0.7312716041249101 mds_error 0.0
Combined_error: 1.4625432082498202
Contribution: 228

Price 707
semantic_error 0.731570464894554 mds_error 0.0
Combined_error: 1.463140929789108
Contribution: 237

Price 721
semantic_error 0.7319364246656095 mds_error 0.0
Combined_error: 1.463872849331219
Contribution: 246

Price 736
semantic_error 0.7324028691244049 mds_error 0.0
Combined_error: 1.4648057382488098
Contribution: 255

Price 750
semantic_error 0.732907479081958 mds_error 0.0
Combined_error: 1.465814958163916
Contribution: 264



In [66]:
new_features_more = meow_feat # Change price

semantic_regressed_more = np.array(solver.regress_semantic(new_features_more))
semantic_regressed_more = [round(sem, 1) for sem in semantic_regressed_more]
semantic_regressed_more

[4.3, 2.6, 4.6, 4.3, 4.1, 5.3]

In [67]:
explorers_semantic_mk

Resolution           4.61
Energy Efficiency     4.8
Carbon Footprint     2.61
Connectivity         3.99
No. of Apps          3.89
Price                4.43
Name: Followers_8, dtype: object

In [68]:
an.compute_distance_centroids(semantic_regressed_more, explorers_semantic_mk)[3]

{'observation': {'centroid': 0.8189001692634407}}

In [69]:
an.compute_distance_centroids(semantic_regressed_more, explorers_semantic_mk)[4]

{'observation': {'centroid': array([0.3100000000000005, 2.1999999999999997, 1.9900000000000002,
         0.31000000000000005, 0.20999999999999952, 0.8699999999999992],
        dtype=object)}}

In [70]:
mds_more = solver.regress_mds(new_features_more)
mds_more = [round(mds) for mds in mds_more] 
mds_more

[0, 0, 0]

In [71]:
an.compute_distance_centroids(mds_more, explorers_mds_mk, weighted="eq", max_distance_1D=40)[3]

d:\Uni\Anno III\UCSD\112\Markstrat\Code\Utils.py:271: RuntimeWarning: invalid value encountered in divide
  relative_distance = manhattan_distance / feat_values_array
d:\Uni\Anno III\UCSD\112\Markstrat\Code\Utils.py:283: RuntimeWarning: invalid value encountered in divide
  relative_metric = (current_centroid - feat_values_array)/feat_values_array


{'observation': {'centroid': 1.0}}

In [72]:
brd.move_mds

,Autonomy,Sophistication,Economy
MARKET : Vodites,,,


In [73]:
explorers_mds_mk

[0, 0, 0]

In [74]:
an.compute_distance_centroids(mds_more, explorers_mds_mk, weighted="eq", max_distance_1D=40)[4]

d:\Uni\Anno III\UCSD\112\Markstrat\Code\Utils.py:271: RuntimeWarning: invalid value encountered in divide
  relative_distance = manhattan_distance / feat_values_array
d:\Uni\Anno III\UCSD\112\Markstrat\Code\Utils.py:283: RuntimeWarning: invalid value encountered in divide
  relative_metric = (current_centroid - feat_values_array)/feat_values_array


{'observation': {'centroid': array([0, 0, 0], dtype=int64)}}